In [3]:
import keras
from keras.models import Sequential,Model
from keras.layers.core import Permute

In [10]:
model = Sequential()
model.add(Permute((2, 1), input_shape=(10, 64)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_7 (Permute)          (None, 64, 10)            0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
